# 4.0 Read mapping and coverage

## Software and versions used in this study

- BBMap v39.01
- SAMtools v1.19
- featureCounts (subread-2.0.6-Linux-x86_64)

## Additional custom scripts

Note: custom scripts have been tested in python v3.11.6 and R v4.2.1 and may not be stable in other versions.

- scripts/general/featurecounts_make_feature_table.py
- scripts/general/summarise_counts.py
- scripts/general/summarise_counts.R

*Required python packages: argparse, pandas, numpy, re, pathlib, subprocess, os*

*Required R libraries: dplyr, tibble, readr, tidyr, fuzzyjoin, stringr, matrixStats, edgeR, EDASeq*

***

## Read mapping and abundance

Map DNA and RNA sequencing reads to dereplicated genomes and calculate abundances across samples. 

Note: 

- If you have prokaryote and/or eukaryote metagenome-assembled genomes available from the same dataset, it is preferable to include these together with vOTUs when generating the read mapping index to limit mis-mapping of non-virus-derived reads to similar genomic regions in viruses (e.g. AMGs derived from host genomes)

#### Prep MAG contigIDs:

- If necessary, modify scaffold names within each MAG fna file to include file name (i.e. magID) to assist identifying individual genomes in downstream analyses

In [ ]:
for file in DNA/2.prokaryote_MAGs/4.bin_dereplication_dRep/1.dRep_out/dereplicated_genomes/*.fa; do
    magID=$(basename ${file} .fa)
    sed -i -e "s/>/>${magID}_/g" ${file}
done


#### Concatenate MAGs with vOTUs

In [ ]:
mkdir -p DNA/4.read_mapping
cat DNA/3.viruses/5.checkv_vOTUs/vOTUs.fna DNA/2.prokaryote_MAGs/4.bin_dereplication_dRep/1.dRep_out/dereplicated_genomes/*.fa > DNA/4.read_mapping/vOTU_and_MAG_contigs.fna

#### Read mapping: build reference index


In [ ]:
cd DNA/4.read_mapping/
bbmap.sh -Xmx24g ref=vOTU_and_MAG_contigs.fna

### Whole-genome sequencing (WGS)

#### WGS: read mapping

Note: Read mapping is set to 95% identity (`minid = 0.95`) in the example below due to vOTUs being clustered at 95% sequence similarity (over >= 85% sequence length). Modify as necessary.


In [ ]:
cd DNA/4.read_mapping/
mkdir -p WGS

for i in {1..9}; do
    # Read mapping
    bbmap.sh \
    t=30 -Xmx140g ambiguous=best minid=0.95 \
    in1=../1.Qual_filtered_trimmomatic/S${i}_R1.fastq \
    in2=../1.Qual_filtered_trimmomatic/S${i}_R2.fastq \
    covstats=WGS/S${i}.covstats.txt \
    statsfile=WGS/S${i}.statsfile.txt \
    out=WGS/S${i}.sam
    # convert to bam
    samtools sort -@ 10 -o WGS/S${i}.bam WGS/S${i}.sam
    # Pileup (from BBMap tools)
    pileup.sh in=WGS/S${i}.sam rpkm=WGS/S${i}.covstats_pileup.txt
    # optional clean up
    rm WGS/Filt.S${i}.sam
done

#### WGS: Compile counts table of read counts per contig per sample

Note:

- The companion script *summarise_counts.R* must be located in a directory available in your PATH variable (Alternatively, you can add the scripts directory to $PATH via `export PATH="/path/to/scripts/general/:$PATH"`)
- To input multiple files (i.e. read mapping from multiple samples), the quote marks are necessary with `--input '*.covstats_pileup.txt'` for the wildcard to be interpreted correctly.

In [ ]:
scripts/general/summarise_counts.py \
--input 'DNA/4.read_mapping/WGS/*.covstats_pileup.txt' --format pileup \
--lib_norm total \
--count_threshold 10 \
--read_counts DNA/4.read_mapping/WGS/wgs.summary_read_counts.tsv \
--output DNA/4.read_mapping/WGS/wgs.summary_count_table.tsv


### Whole-transcriptome sequencing (WTS)

Map trimmed and filtered (incl rRNA removal) RNA reads to the same index generated above to calculate feature (e.g. gene) abundance (i.e. transcription)

#### WTS: read mapping

In [ ]:
cd DNA/4.read_mapping/
mkdir -p WTS/

for i in {1..9}; do
    # Read mapping
    bbmap.sh \
    t=30 -Xmx90g ambiguous=best minid=0.95 \
    in1=../../RNA/2.rRNA_filtered/unaligned/S${i}_non_rRNA_fwd.fq \
    in2=../../RNA/2.rRNA_filtered/unaligned/S${i}_non_rRNA_rev.fq \
    covstats=WTS/S${i}.covstats.txt \
    statsfile=WTS/S${i}.statsfile.txt \
    out=WTS/S${i}.sam
    # convert to bam
    samtools sort -@ 20 -o WTS/S${i}.bam WTS/S${i}.sam
    # pileup 
    pileup.sh in=WTS/S${i}.sam rpkm=WTS/S${i}.covstats_pileup.txt
    # optional clean up
    rm WTS/S${i}.sam
done

#### WTS: Generate feature table (gene_coords) table for featureCounts

Generate a feature table in SAF format (required for featureCounts)

You can pull this information from *prodigal* or *DRAM/DRAM-v* output files. The script below generates an SAF formatted feature table from DRAM (prokaryotes) and/or DRAMv (viruses) annotations.tsv, rrna.tsv, and trna.tsv data.

In [ ]:
scripts/general/featurecounts_make_feature_table.py \
-a DNA/2.prokaryote_MAGs/5.gene_annotation/1.dram_annotate_dRep_mags/collated_dram_annotations.tsv \
-t DNA/2.prokaryote_MAGs/5.gene_annotation/1.dram_annotate_dRep_mags/collated_dram_trnas.tsv \
-r DNA/2.prokaryote_MAGs/5.gene_annotation/1.dram_annotate_dRep_mags/collated_dram_rrnas.tsv \
-x DNA/3.viruses/7.gene_annotation/dramv_annotation/collated_dramv_annotations.tsv \
-y DNA/3.viruses/7.gene_annotation/dramv_annotation/collated_dramv_trnas.tsv \
-z DNA/3.viruses/7.gene_annotation/dramv_annotation/collated_dramv_rrnas.tsv \
-o DNA/4.read_mapping/WTS/gene_coords.SAF

#### WTS: Run featureCounts

Assign mapped RNA reads to features (e.g. genes) via featureCounts


In [ ]:
mkdir -p DNA/4.read_mapping/WTS/featureCounts

featureCounts \
-T 8 -p --countReadPairs -t exon -F SAF \
-a DNA/4.read_mapping/WTS/gene_coords.SAF \
-o DNA/4.read_mapping/WTS/featureCounts/wts.gene_counts.txt \
DNA/4.read_mapping/WTS/*.bam


#### WTS: Generate sample mapping file

Create sample mapping file. 

Note: this is required for *summarise_counts.py* to summarise featureCounts data. This expects a tsv file with the following colums: 

- sampleID: unique filename substrings (one per file) that identify the sample
- group: group or category that that sample belongs to (e.g. 'freshwater', 'marine') (this is used if calculating TMM and/or edgeR stats)
- lib.size: total library size (read counts) of the sample. Note, this can be taken from the sample's statsfile.txt generated during BBMap readmapping (in the first line: "Reads Used: <library_size>")

Example:

| sampleID | group | lib.size |
| --- | --- | --- |
| S1 | freshwater | 15800068 |
| S2 | freshwater | 30547744 |
| S3 | freshwater | 18567630 |
| S4 | marine | 15753862 |
| S5 | marine | 11432528 |
| S6 | marine | 18701302 |

#### WTS: feature (gene) coverage

Note: The companion script *summarise_counts.R* must be located in a directory available to your PATH variable (Alternatively, you can add the scripts directory to $PATH via `export PATH="/path/to/scripts/general/:$PATH"`)

In [ ]:
scripts/general/summarise_counts.py \
--input DNA/4.read_mapping/WTS/featureCounts/wts.gene_counts.txt --format featurecounts \
--sample_mapping_file DNA/4.read_mapping/WTS/wts_sample_mapping_file.txt \
--lib_norm total \
--count_threshold 5 \
--read_counts DNA/4.read_mapping/WTS/wts.summary_read_counts.tsv \
--output DNA/4.read_mapping/WTS/wts.summary_count_table.tsv

***